In [22]:
# 安裝yahoo finance套件
!pip install yfinance

In [23]:
import google.generativeai as genai  # Google生成式AI套件Gemini API
import os  # 系統檔，用來操作檔案與文件
import requests  # 用來發HTTP請求，用於網路爬蟲
from bs4 import BeautifulSoup  # 網路爬蟲套件
import numpy as np  # 數值計算
import pandas as pd  # 資料處理
import yfinance as yf  # Yahoo finance 股價資訊
import datetime as dt  # 處理日期時間
import time  # 時間延遲套件，用於控制API呼叫頻率


# 爬取股票資訊的類別
class StockInfo():
    """
    股票基本資訊爬取類別

    功能：
    1. 從台灣證券交易所爬取所有上市公司的股號、股名、產業別
    2. 提供股號查詢股名的功能

    資料來源：台灣證券交易所 (TWSE) ISIN網站
    """

    # 取得全部股票的股號、股名
    def stock_name(self):
        """
        爬取所有台股的股號、股名、產業別

        Returns:
            DataFrame: 包含「股號」、「股名」、「產業別」三欄的資料表

        技術說明：
        - 使用requests取得網頁內容
        - 使用BeautifulSoup解析HTML
        - 過濾出4碼股號的上市公司（排除ETF等其他商品）
        """
        # 向台灣證券交易所發送請求
        response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
        # 使用BeautifulSoup解析HTML內容
        url_data = BeautifulSoup(response.text, 'html.parser')
        # 找出所有的表格列（tr標籤）
        stock_company = url_data.find_all('tr')

        # 資料處理：提取股號、股名、產業別
        # \u3000 是全形空白字元，用來分隔股號和股名
        # 只保留4碼的股號（過濾掉ETF、特別股等）
        data = [
            (row.find_all('td')[0].text.split('\u3000')[0].strip(),
                row.find_all('td')[0].text.split('\u3000')[1],
                row.find_all('td')[4].text.strip())
            for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
        ]
        # 建立DataFrame並設定欄位名稱
        df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])
        return df

    # 取得股票名稱
    def get_stock_name(self, stock_id, name_df):
        """
        根據股號查詢股名

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            name_df (DataFrame): 包含股號和股名的資料表

        Returns:
            str: 股票名稱（例如：'台積電'）
        """
        return name_df.set_index('股號').loc[stock_id, '股名']

In [24]:
class StockAnalysis():
    """
    股票分析類別 - 整合股價資料與Gemini AI分析

    主要功能：
    1. 取得股票價格資料（技術面）
    2. 取得股票基本面資料（營收、EPS）
    3. 使用Gemini AI進行股票分析
    4. 多檔股票比較與推薦

    設計理念：
    - 整合多元數據源（價格、基本面）
    - 利用LLM進行智能分析
    """

    def __init__(self, gemini_api_key):
        """
        初始化股票分析系統

        Args:
            gemini_api_key (str): Google Gemini API金鑰

        初始化步驟：
        1. 設定Gemini API
        2. 建立生成模型實例
        """
        # 設定Gemini API金鑰
        genai.configure(api_key=gemini_api_key)
        """
        1. Gemini 2.5 Pro - 進階思考模型
        2. Gemini 2.5 Flash - 快速且智慧
        3. Gemini 2.5 Flash-Lite - 速度最快的 Flash 模型，可提高成本效益和輸送量。
        """
        # 使用Gemini 2.5 Pro模型（高階推理能力）
        self.model = genai.GenerativeModel('gemini-2.5-pro')
        # 建立股票資訊物件
        self.stock_info = StockInfo()
        # 預先載入所有股票名稱（避免重複爬取）
        self.name_df = self.stock_info.stock_name()


    def stock_price(self, stock_id, days=15):
        """
        取得股票近期價格資料（技術面分析基礎）

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            days (int): 要取得的天數（預設15天）

        Returns:
            dict: 包含日期、收盤價、每日報酬率的字典

        技術說明：
        - 使用yfinance API取得Yahoo Finance資料
        - 計算每日報酬率 = (今日收盤-昨日收盤)/昨日收盤
        - 自動加上.TW後綴（台股代碼格式）
        """

        # 台股代碼需要加上.TW後綴
        stock_id += '.TW'
        # 計算日期區間
        end = dt.date.today() # 今天
        start = end - dt.timedelta(days=days) # 往前推N天

        # 禁用 FutureWarning
        import warnings
        warnings.filterwarnings('ignore', category=FutureWarning)

        # 下載股價資料
        # auto_adjust=True 會自動調整除權息的價格
        df = yf.download(stock_id, start=start, auto_adjust=True)

        # 將欄位名稱改為中文（方便閱讀）
        df.columns = ['開盤價', '最高價', '最低價', '收盤價', '成交量']
        # 建立回傳的資料結構
        data = {
            '日期': df.index.strftime('%Y-%m-%d').tolist(), # 格式化日期
            '收盤價': df['收盤價'].tolist(), # 收盤價列表
            '每日報酬': df['收盤價'].pct_change().tolist() # 報酬率 = 價格變化百分比
        }
        return data


    def stock_fundamental(self, stock_id):
        """
        取得股票基本面資料（價值投資分析基礎）

        Args:
            stock_id (str): 股票代號

        Returns:
            dict: 包含季度日期、營收成長率、EPS、EPS季增率

        基本面指標說明：
        - 營收成長率：季度營收相比上一季的成長百分比
        - EPS (每股盈餘)：公司獲利能力的核心指標
        - EPS季增率：EPS相比上一季的成長百分比

        注意事項：
        - 某些股票可能無基本面資料（新上市或資料不全）
        - 使用try-except確保程式不會因單一股票錯誤而中斷
        """

        stock_id += '.TW'
        stock = yf.Ticker(stock_id)

        try:
            # 計算季度營收成長率
            # pct_change(-1)：與前一期（-1）比較的百分比變化
            # fill_method=None：不填補缺失值
            quarterly_revenue_growth = np.round(
                stock.quarterly_financials.loc['Total Revenue'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得季度EPS
            quarterly_eps = np.round(
                stock.quarterly_financials.loc['Basic EPS'].dropna().tolist(),
                2
            )
            # 計算EPS季增率
            quarterly_eps_growth = np.round(
                stock.quarterly_financials.loc['Basic EPS'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得日期並格式化
            dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]
            # 組合資料（確保長度一致）
            data = {
                '季日期': dates[:len(quarterly_revenue_growth)],
                '營收成長率': quarterly_revenue_growth.tolist(),
                'EPS': quarterly_eps.tolist(),
                'EPS 季增率': quarterly_eps_growth.tolist()
            }
        except Exception as e:
            # 如果取得資料失敗，回傳空資料
            print(f"無法取得 {stock_id} 基本面資料: {e}")
            data = {'季日期': [], '營收成長率': [], 'EPS': [], 'EPS 季增率': []}

        return data

    def _get_reply(self, content_msg):
        """
        呼叫Gemini API並處理各種回應狀況（內部方法）

        Args:
            content_msg (str): 要傳送給Gemini的提示詞（prompt）

        Returns:
            str: Gemini的回應文字或錯誤訊息

        錯誤處理機制：
        1. finish_reason = 1 (STOP)：正常完成
        2. finish_reason = 2 (MAX_TOKENS)：超過長度限制
        3. finish_reason = 3 (SAFETY)：觸發安全政策
        4. finish_reason = 4 (RECITATION)：引用問題
        5. HTTP 429：API速率限制（免費版：每分鐘5次，每天25次）
        6. HTTP 503：服務暫時不可用
        """

        try:
            # 呼叫Gemini API生成內容
            response = self.model.generate_content(
                content_msg,
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=10000,  # 最大輸出token數（避免過長）
                    temperature=1.0,  # 溫度參數（1.0 = 較有創意，0.0 = 較保守）
                )
            )

            # 檢查是否有回應候選
            if not response.candidates:
                return "模型未返回任何回應"

            candidate = response.candidates[0]
            finish_reason = candidate.finish_reason

            # finish_reason是整數值，代表不同的完成狀態：
            # 0 = UNSPECIFIED (未指定)
            # 1 = STOP (正常完成)
            # 2 = MAX_TOKENS (達到token限制)
            # 3 = SAFETY (安全過濾)
            # 4 = RECITATION (引用問題)
            # 5 = OTHER (其他原因)

            # 根據不同的完成原因處理
            if finish_reason == 1:  # STOP - 正常完成
                # 正常完成
                return candidate.content.parts[0].text

            elif finish_reason == 2:  # MAX_TOKENS - 達到長度限制
                try:
                    partial_text = candidate.content.parts[0].text if candidate.content.parts else ""
                    return f"{partial_text}\n\n [回應被截斷，已達到長度限制]"
                except:
                    return "回應超過長度限制。建議：1)增加 max_output_tokens 2)簡化 prompt"

            elif finish_reason == 3:  # SAFETY - 安全政策
                return "模型因安全政策無法提供回應"

            elif finish_reason == 4:  # RECITATION - 引用問題
                return "回應因引用問題被過濾"

            else:
                return f"未知完成原因: {finish_reason}"

        except Exception as e:
            # 處理API錯誤
            error_msg = str(e)
            # 速率限制錯誤（最常見）
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                return "達到 API 速率限制（免費層：5次/分鐘，25次/天）"
            # 服務暫時不可用
            elif "503" in error_msg:
                return "服務暫時不可用，請稍後重試"
            # 其他錯誤
            else:
                return f"API 錯誤: {error_msg}"


    def stock_gimini_analysis(self, stock_id):
        """
        使用Gemini AI分析單支股票（核心功能）

        Args:
            stock_id (str): 股票代號

        Returns:
            str: Gemini生成的股票分析報告

        分析流程：
        1. 取得股票名稱
        2. 收集技術面資料（價格、報酬率）
        3. 收集基本面資料（營收、EPS）
        4. 將資料摘要後傳送給Gemini
        5. 讓Gemini扮演金融分析師角色，產生專業分析

        Prompt設計要點：
        - 角色設定：明確告訴模型扮演「金融分析師」
        - 數據提供：精簡但完整的關鍵資訊
        - 輸出要求：技術面+基本面+風險評估
        - 語言指定：繁體中文
        """
        # 取得股票名稱
        stock_name = self.stock_info.get_stock_name(stock_id, self.name_df)

        # 收集股價資料
        price_data = self.stock_price(stock_id)
        print('資料收集完畢')
        # 收集基本面資料
        stock_value_data = self.stock_fundamental(stock_id)

        # --- 資料摘要製作 ---
        # 目的：將大量資料濃縮成關鍵指標，避免超過token限制
        summary_parts = []

        # 股價摘要：最新價格 + 兩週漲跌幅
        if price_data and price_data['收盤價']:
            latest = price_data['收盤價'][-1] # 最新收盤價
            first = price_data['收盤價'][0] # 首日收盤價
            change = ((latest - first) / first) * 100 # 計算漲跌百分比
            summary_parts.append(f"股價：{latest:.2f} (兩週 {change:+.2f}%)")

        # 基本面摘要：最新一季的營收成長和EPS
        if stock_value_data and stock_value_data['季日期']:
            q = stock_value_data['季日期'][0] # 季度
            rev = stock_value_data['營收成長率'][0] # 營收成長率
            eps = stock_value_data['EPS'][0] # EPS
            summary_parts.append(f"基本面({q})：營收成長{rev*100:.1f}%, EPS {eps:.2f}")

        # 組合所有摘要
        combined = "；".join(summary_parts)

        # ============ 作業修改區 START ============
        # --- 建構Prompt ---
        # 這是Prompt Engineering的關鍵：如何引導LLM產生專業分析
        """
        'conservative': '''你是保守型財務顧問...
        1. 優先評估下檔風險
        2. 關注股息穩定性
        3. 避免高波動股票...''',

        'aggressive': '''你是成長型投資顧問...
        1. 關注營收成長動能
        2. 尋找突破訊號...''',

        'balanced': '''你是專業金融分析師...
        平衡評估機會與風險...'''
        """
        # 取得當前日期
        current_date = dt.date.today().strftime('%Y年%m月%d日')

        content_msg = f'''你是一位具備CFA等級分析能力的投資研究員，原則：
1. 報告需包含 EPS、毛利率、ROE、自由現金流等財務指標。
2. 同時分析 Top-down（全球/產業）與 Bottom-up（公司）。
3. 評估合理估值（PE、PB、EV/EBITDA）。
4. 給予「中性＋數據化」建議，而非感覺派。
報告日期請使用當前日期：{current_date}。
分析股票：{stock_name}。
資料：{combined}

    請提供詳細的技術面和基本面見解，並考慮風險因素。繁體中文回應。'''
        # ============ 作業修改區 END ============
        # 呼叫Gemini API
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_sort(self, message):
        """
        讓Gemini為多檔股票評分並排序

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini生成的評分排序結果

        評分標準：
        - 50分為基準
        - 正面因素加分（例如：營收成長、技術面轉強）
        - 負面因素扣分（例如：虧損、下跌趨勢）
        - 綜合技術面+基本面+市場環境

        應用場景：投資組合篩選、資金配置決策
        """

        content_msg = f'''你是專業股票分析師，根據趨勢報告評分(0-100)。
50分為基準，正面消息加分，負面消息扣分。
排序所有股票。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_choice(self, message):
        """
        讓Gemini從多檔股票中選出最佳投資標的

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini選出的最佳股票及理由

        決策邏輯：
        - 即使所有選項都不理想，也要選出相對最佳的
        - 需要說明選擇理由（風險、報酬、時機等）
        - 模擬真實的投資決策情境

        教學價值：
        - 理解LLM如何進行多維度評估
        - 學習投資決策的思考框架
        """

        content_msg = f'''你是專業證券分析師，從趨勢報告中選出最適合投資的一檔股票。
即使都不理想也要選一檔，說明理由。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply

In [25]:
# 【重要】請修改為你自己的Gemini API Key
# ============ 作業修改區 START ============
gemini_api_key = '我的API Key'
# ============ 作業修改區 END ============
# 建立股票分析物件
stock_analysis = StockAnalysis(gemini_api_key)

# 單支股價分析

In [26]:
# ============ 作業修改區 START ============
# 改成想分析的股票代號
reply = stock_analysis.stock_gimini_analysis('2308')
# ============ 作業修改區 END ============
print(reply)

[*********************100%***********************]  1 of 1 completed


資料收集完畢
好的，這是一份依據您指定原則撰寫的台達電子 (2308.TW) 投資研究報告。

***

### **台達電子 (2308.TW) 投資研究報告**

**報告日期：** 2025年11月20日
**分析師：** [CFA Charterholder]
**目前股價：** NT$ 939.00 (截至 2025年11月19日)
**目標價 (12個月)：** NT$ 965.00
**評級：中性 (Neutral)**

---

#### **1. 投資摘要 (Executive Summary)**

台達電子作為全球電源管理與散熱解決方案的領導者，正處於全球AI、電動車(EV)與綠色能源轉型的核心位置。公司憑藉其深厚的技術護城河與多元化的產品組合，長期成長趨勢明確。

然而，我們觀察到截至2025年中期的營收成長呈現溫和（約4.0%），顯示部分終端市場（如消費性電子、傳統工業自動化）可能面臨週期性挑戰。目前的股價 (NT$ 939) 已高度反映了市場對其在AI伺服器電源與散熱領域的樂觀預期，估值處於歷史區間上緣。

基於我們的分析，台達電的長期基本面依然強勁，但短期內高估值限制了股價的上行潛力。因此，我們給予**「中性 (Neutral)」**評級，建議投資人等待更具吸引力的估值水平或更強勁的成長催化劑出現。

---

#### **2. Top-down 分析：總體經濟與產業趨勢**

*   **全球宏觀環境 (2025下半年展望):**
    全球主要經濟體預計將維持緩步增長，高利率環境的影響逐漸顯現，企業資本支出趨於謹慎。然而，針對特定領域的財政刺激政策（如美國《通膨削減法案》IRA、歐洲綠色新政）持續為綠能、儲能及電動車基礎設施提供結構性支撐。

*   **產業趨勢分析:**
    1.  **AI伺服器與資料中心：** 這是台達電目前最強勁的成長動能。隨著大型語言模型(LLM)與生成式AI應用普及，對高功率、高效率的電源供應器(PSU)與先進散熱解決方案（如液冷技術）的需求呈現指數級增長。預計2026年後，AI伺服器電源市場年複合成長率(CAGR)將超過30%，台達電作為主要供應商將直接受益。
    2.  **電動車 (EV)：** 全球EV滲透率持續提升，但增速可能因經濟放緩及高基期而趨於平穩。台達電在車載充電器(OB

# 收集多檔股票的趨勢報告

In [27]:
# 建立股票清單
# ============ 作業修改區 START ============
# 請修改為你想分析的股票清單（建議3-5檔，避免超過API限制）
# 台股股票參考：
# 0050 前十大持股參考：
# 2330 台積電、2317 鴻海、2454 聯發科、2308 台達電、3711 日月光投控
# 2382 廣達、2881 富邦金、2891 中信金、2882 國泰金、2303 聯電
stock_list = ['2330', '2454', '2308']
# ============ 作業修改區 END ============

# 設定儲存路徑
today_time = dt.date.today().strftime('%Y%m%d')
path = './StockGemini/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

print(f"\n{'='*70}")
print(f"開始分析 {len(stock_list)} 檔股票")
print(f"{'='*70}\n")

# 批次處理每一檔股票
for i, stock in enumerate(stock_list, 1):
    print(f"[{i}/{len(stock_list)}] 處理股票: {stock}")

    file_path = f"{path}trend_{stock}_{today_time}.txt"

    if not os.path.exists(file_path):
        try:
            # 分析股票
            print(f"   正在分析...")
            analysis = stock_analysis.stock_gimini_analysis(stock_id=stock)

            # 儲存結果
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(analysis)

            content[stock] = analysis
            print(f"   分析完成並已儲存\n")

            # 速率限制控制
            if i < len(stock_list):
                wait_time = 32
                print(f"   等待 {wait_time} 秒以符合 API 速率限制...\n")
                time.sleep(wait_time)

        except Exception as e:
            error_msg = f"錯誤: {str(e)}"
            print(f"   {error_msg}\n")
            content[stock] = error_msg

    else:
        # 讀取已存在的分析
        with open(file_path, "r", encoding="utf-8") as f:
            content[stock] = f.read()
        print(f"   從快取讀取\n")

print(f"{'='*70}")
print("所有股票分析完成！")
print(f"{'='*70}\n")

print("\n" + "="*70)
print("分析結果")
print("="*70 + "\n")

for stock_id, analysis in content.items():
    try:
        stock_name = stock_analysis.stock_info.get_stock_name(stock_id, stock_analysis.name_df)
    except KeyError:
        stock_name = stock_id # Fallback to using stock_id as name if not found
        print(f"警告: 股票代號 {stock_id} 的名稱未在資料中找到。")

    print("="*70)
    print(f"【{stock_id} - {stock_name}】")
    print("="*70)
    print(analysis)
    print("\n")


開始分析 3 檔股票

[1/3] 處理股票: 2330
   從快取讀取

[2/3] 處理股票: 2454
   從快取讀取

[3/3] 處理股票: 2308
   從快取讀取

所有股票分析完成！


分析結果

【2330 - 台積電】
好的，這是一份依據您提供的資訊與要求，以CFA等級投資研究員角度撰寫的台積電（2330.TW）投資分析報告。

---

### **台積電 (TSMC, 2330.TW) 投資分析報告**

**報告日期：** 2024年5月24日
**當前股價：** TWD 1,450.00
**評級建議：** 中性偏多 (Neutral-Positive)

---

### **1. 投資摘要 (Executive Summary)**

台積電作為全球晶圓代工的絕對領導者，其在先進製程的技術護城河無可撼動，是人工智慧（AI）與高效能運算（HPC）趨勢下的核心受惠者。

*   **基本面：** 公司營運由AI需求強勁驅動，抵銷了部分消費性電子需求的疲軟。預期2025年將維持穩健的營收與獲利增長。財務結構極為健全，ROE與自由現金流創造能力維持頂尖水準。
*   **估值面：** 當前股價已反映市場對其AI領導地位的高度預期，預估本益比（Forward P/E）處於歷史區間上緣。雖長期價值明確，但短期估值已不便宜。
*   **技術面：** 股價處於長期多頭趨勢，但短期漲多後出現高檔震盪，呈現量縮整理格局，顯示市場在追價上態度轉趨謹慎。
*   **風險面：** 地緣政治為最大不確定性因素，其次為半導體產業的週期性、單一客戶集中度以及海外擴廠的成本與效率挑戰。

綜合評估，台積電的長期基本面與產業地位依然強勁，但在當前估值水平下，股價的上檔空間可能與未來一年的業績增長相匹配，而非大幅超越。建議採「**逢回檔分批佈局**」策略，而非積極追高。

---

### **2. 由上而下分析 (Top-down Analysis)**

#### **2.1 全球總體經濟**
*   **利率環境：** 美國聯準會（Fed）利率政策仍是市場焦點。若通膨數據持續降溫，降息預期將有利於科技股等成長型資產的估值擴張。反之，若利率維持高檔，將對高估值股票造成壓力。
*   **經濟增長：** 全球主要經濟體增長力道不一，但AI所帶動的資本支出是

In [28]:
reply = stock_analysis.stock_gimini_choice(str(content))
print(reply)

好的，身為專業證券分析師，我已詳細審閱您提供的三份趨勢報告。我的任務是，即使都不完美，也要從中選擇一檔現階段最適合投資的股票，並闡述我的決策邏輯。

綜合評估這三份趨勢報告，我會選擇 **聯發科 (2454) 為最適合的投資標的**。

以下是我的詳細分析與決策過程：

### **總體評估框架**

我的評選標準基於三個核心維度：

1.  **基本面強度與成長故事 (Fundamental Strength & Growth Story):** 公司的護城河、產業地位及未來成長動能。
2.  **估值合理性 (Valuation):** 目前的股價是否昂貴？是否已過度反映利多？是否存在安全邊際？
3.  **風險報酬比 (Risk/Reward Ratio):** 結合前兩者，評估當下投入資金所能期待的潛在回報與需承擔的潛在風險。

### **三檔個股橫向比較**

| 評估維度 | **台積電 (2330)** | **聯發科 (2454)** | **台達電 (2308)** |
| :--- | :--- | :--- | :--- |
| **核心護城河** | **極強** (技術壟斷) | **強** (市場份額、IP組合) | **強** (技術、客戶關係) |
| **成長故事** | **清晰且強勁** (AI晶片需求) | **清晰且多元** (AI手機, AI PC, 車用) | **極具吸引力** (AI散熱, EV, 儲能) |
| **估值水平** | **偏高** (處於歷史區間上緣) | **相對合理** (處於歷史區間中上緣) | **最高** (已高於歷史平均) |
| **報告揭示風險** | 地緣政治、估值壓力 | 短期技術面回檔、競爭 | 估值已先行、部分業務疲軟 |
| **分析師評級** | 中性偏多 | 中性 | 中性持有 |

---

### **最終決策理由**

我的選擇是 **聯發科 (2454)**，主要基於以下幾點考量，它在當前時點提供了最佳的「**平衡性**」。

#### **1. 估值最具吸引力，提供了較佳的安全邊際**

這是最關鍵的決策點。三家都是頂尖公司，但投資不僅是買好公司，更是要買在「好價格」。

*   **台達電 (2308):** 報告明確指出其2025年預估本益比(25.3倍

# 推薦股票的評分排序

In [29]:
reply = stock_analysis.stock_gimini_sort(str(content))
print(reply)

好的，我將以專業股票分析師的角色，根據您提供的詳細趨勢報告，對這三家公司進行評分與排序。

評分標準如下：
*   **基準分：** 50分 (中性)
*   **加分項：** 產業龍頭地位、強勁成長動能、優異的財務指標、技術面偏多、估值具吸引力。
*   **扣分項：** 嚴峻的競爭環境、成長趨緩、財務指標惡化、技術面轉弱、估值偏高、顯著風險因子。

---

### **趨勢報告評分總覽**

| 股票代號 | 公司名稱 | 趨勢報告評分 | 核心評語 |
| :--- | :--- | :---: | :--- |
| **2308** | **台達電** | **65** | **多引擎成長趨勢明確，財務結構優化，惟估值已高。** |
| **2330** | **台積電** | **63** | **AI霸主地位無可撼動，但估值與地緣政治風險限制了評分。** |
| **2454** | **聯發科** | **45** | **基本面自谷底回升，但短期技術面轉弱且估值已反映預期。** |

---

### **個股分析與評分詳解**

#### **第一名：2308 台達電 (Delta Electronics)**

**最終評分：65 / 100**

**基準分 (50) + 加分項 (20) - 扣分項 (5) = 65**

**綜合評語：**
台達電是一家基本面極其優異的公司，完美卡位AI、電動車、儲能三大長期趨勢，形成了強大的「多成長引擎」。報告中明確指出其毛利率持續改善，證明其高價值策略奏效。技術面也呈現健康的多頭排列。雖然估值偏高是主要限制，但相較於另外兩家公司，其成長故事的多元性與穩健性給予了更高的分數。

*   **加分項目 ( +20分 )**
    *   **多元成長動能 (+10):** 報告強調公司是AI伺服器、電動車(EV)及儲能「多重長期增長趨勢的核心」，業務佈局廣泛且前景清晰，降低了對單一產業的依賴。
    *   **財務指標優化 (+5):** 報告預估其毛利率將從28.9%持續改善至31.0%，ROE也長期維持在20%以上，顯示其盈利能力與經營效率正在提升。
    *   **技術面健康 (+5):** 報告指出股價處於「長期上升趨勢」且為「多頭排列」，顯示市場動能依然強勁。

*   **扣分項目 